## Segmentation

### Mount Drive

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Importing Packages

In [ ]:
# Fetching Dataset
import pandas as pd

# Preprocessing Image
import cv2 as cv
# import matplotlib.pyplot as plt
# from google.colab.patches import cv2_imshow

# Segmenting Image
import numpy as np
import math

### Fetching Dataset

In [ ]:
dataset_dir = "/content/drive/MyDrive/Dataset/Dataset_Thesis/"
# dataset_dir = "../Dataset_Thesis/"
dataset = pd.read_csv(dataset_dir + "BD_DB_64/BD_DB_64_raw.csv")

In [ ]:
dataset

,Writer,Filename,Gender,Age,Collector,Collection Date,Ratio,Unnamed: 7,Unnamed: 8,Subtotal,Total
0,Avishek Chowdhury,AC_01.jpg,Male,24,Avishek Chowdhury,"21, Mar 2023",Gender,Male,53.85%,14.0,26.0
1,Anik Barua,AB_01.jpg,Male,25,Anik Barua,"21, Mar 2023",NaN,Female,46.15%,12.0,NaN
2,Sathi Barua,AB_03.jpg,Female,25,Anik Barua,"21, Mar 2023",Age,10-24,42.31%,11.0,26.0
3,Roni Rani Barua,AB_04.jpeg,Female,30,Anik Barua,"21, Mar 2023",NaN,25 - 39,42.31%,11.0,NaN
4,Roni Barua,AB_05.jpg,Male,28,Anik Barua,"21, Mar 2023",NaN,40 +,15.38%,4.0,NaN
5,Shaharia Hossen,AC_02.jpg,Male,14,Avishek Chowdhury,"23, Mar 2023",NaN,NaN,NaN,NaN,NaN
6,Saionty Singha,AC_04.jpg,Female,15,Avishek Chowdhury,"28, Mar 2023",NaN,NaN,NaN,NaN,NaN
7,Archana Singha,AC_05.jpg,Female,65,Avishek Chowdhury,"28, Mar 2023",NaN,NaN,NaN,NaN,NaN
8,Farhan,AH_1.jpg,Male,31,Md Ahasan Hossen,"22, May 2023",NaN,NaN,NaN,NaN,NaN
9,Asma,AH_2.jpg,Female,29,Md Ahasan Hossen,"22, May 2023",NaN,NaN,NaN,NaN,NaN


In [ ]:
X = dataset.iloc[:,1].values
print(X.shape)

(26,)


In [ ]:
# X

### Preprocessing & Segmenting Images

In [ ]:
def resize_image(candidate_image):
  # Resizing Image
  height, weight, channels = candidate_image.shape
  if(weight>height):
    new_weight = 1754
    new_height = 1240 
  elif(height==weight):
    new_weight = 1240 
    new_height = 1240 
  else:
    new_weight = 1240 
    new_height = 1754 
  candidate_rs = cv.resize(candidate_image, dsize=(new_weight, new_height))
  return candidate_rs

In [ ]:
def preprocess_image(candidate_rs):
  # Converting Image to Grayscale
  candidate_gray = cv.cvtColor(candidate_rs, cv.COLOR_BGR2GRAY)
  # Computing a Global Threshold Value from Grayscale Image(Using Otsu's Method) & Converting to Binary Image
  ret, candidate_bw = cv.threshold(candidate_gray, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)
  # Canny Edge Detection
  candidate_median = int(np.median(candidate_gray))
  candidate_low = np.min(candidate_gray)
  candidate_high = np.max(candidate_gray)
  max_threshold = ret
  min_threshold = abs(ret - candidate_median)
  candidate_edged = cv.Canny(candidate_gray, max_threshold, min_threshold)
  return candidate_edged

In [ ]:
def transform_image(distance_between_lines, distance_between_words, candidate_edged):
  # Morphological Transformations
  kernel_lines = np.ones((distance_between_lines, 450), np.uint8)
  dilated_lines = cv.dilate(candidate_edged, kernel_lines, iterations=1)
  kernel_words = np.ones((distance_between_lines, distance_between_words), np.uint8)
  dilated_words = cv.dilate(candidate_edged, kernel_words, iterations=1)
  return dilated_lines, dilated_words

In [ ]:
def segment_lines(img_path, candidate_rs, dilated_lines):
  # Detecting Connected Component
  # Line Segmentation
  output_cc_lines = cv.connectedComponentsWithStats(dilated_lines)
  total_lines, labels_lines, stats_lines, centroids_lines = output_cc_lines
  # Sorting the components
  for j in range(1, total_lines):
    for i in range(2, total_lines):
      if(stats_lines[i][1] < stats_lines[i-1][1]):
          # print(stats_lines[0],stats_lines[1]);
          labels_lines[[i, i-1]] = labels_lines[[i-1, i]]
          stats_lines[[i, i-1]] = stats_lines[[i-1, i]]
          centroids_lines[[i, i-1]] = centroids_lines[[i-1, i]]
          # print(stats_lines[0],stats_lines[1]);
          # print(i)
  total_lines, labels_line, stats_lines, centroids_line = output_cc_lines
  candidate_lines = candidate_rs.copy()
  roi_lines = []
  for i in range(1, total_lines):
    x_line = stats_lines[i, cv.CC_STAT_LEFT]
    y_line = stats_lines[i, cv.CC_STAT_TOP]
    w_line = stats_lines[i, cv.CC_STAT_WIDTH]
    h_line = stats_lines[i, cv.CC_STAT_HEIGHT]
    # a_new = stats_lines[i, cv.CC_STAT_AREA]
    # a_line = w_line * h_line
    # x_new, y_new, w_new, h_new = stats[i][0], stats[i][1], stats[i][2], stats[i][3]
    if(w_line>50 and h_line>50 and h_line<1000):
      cv.rectangle(candidate_lines, (x_line, y_line), (x_line + w_line, y_line + h_line), (255, 0, 0), 3,)
      roi_lines.append([x_line, y_line, x_line + w_line, y_line + h_line])
  line_no = 1
  for line in roi_lines:
      image = candidate_rs[line[1] : line[3], line[0] : line[2]]
      # Saving the processed image
      new_img_name = dataset_dir + "BD_DB_64/segmented/" + "line_" + str(line_no) + "_" + img_path
      cv.imwrite(new_img_name, image)
      line_no += 1
  print(img_path, "(Line):", len(roi_lines), end =" ")

In [ ]:
def segment_words(found_64, img_path, candidate_rs, dilated_words):
  # Word Segmentation
  output_cc_words = cv.connectedComponentsWithStats(dilated_words)
  total_words, labels_words, stats_words, centroids_words = output_cc_words
  # Sorting the components
  for j in range(1, total_words):
    for i in range(2, total_words):
      if((stats_words[i][1] < stats_words[i-1][1] + stats_words[i-1][3]) or (stats_words[i][1] == stats_words[i-1][1] + stats_words[i-1][3])):
        if(stats_words[i][0] < stats_words[i-1][0]):
          # print(stats[0],stats[1]);
          labels_words[[i, i-1]] = labels_words[[i-1, i]]
          stats_words[[i, i-1]] = stats_words[[i-1, i]]
          centroids_words[[i, i-1]] = centroids_words[[i-1, i]]
          # print(stats[0],stats[1]);
          # print(i)
  total_words, labels_words, stats_words, centroids_words = output_cc_words
  candidate_words = candidate_rs.copy()
  roi_words = []
  for i in range(1, total_words):
    x_word = stats_words[i, cv.CC_STAT_LEFT]
    y_word = stats_words[i, cv.CC_STAT_TOP]
    w_word = stats_words[i, cv.CC_STAT_WIDTH]
    h_word = stats_words[i, cv.CC_STAT_HEIGHT]
    # a_word = stats[i, cv.CC_STAT_AREA]
    # a_word = w_word * h_word
    # x_word, y_word, w_word, h_word = stats[i][0], stats[i][1], stats[i][2], stats[i][3]
    if(w_word>50 and w_word<500 and h_word>25 and h_word<200):
      cv.rectangle(candidate_words, (x_word, y_word), (x_word + w_word, y_word + h_word), (0, 255, 0), 3,)
      roi_words.append([x_word, y_word, x_word + w_word, y_word + h_word])
  word_no = 1;
  for word in roi_words:
      image = candidate_rs[word[1]:word[3], word[0]:word[2]]
      # Saving the processed image
      new_img_name = dataset_dir + "BD_DB_64/segmented/" + "word_" + str(word_no) + "_" + img_path
      cv.imwrite(new_img_name, image)
      word_no += 1;
  print(img_path, "(Word):", len(roi_words))
  if(len(roi_words) == 64):
    found_64 += 1
  return found_64

In [ ]:
# Segmentation Criteria
distance_between_lines = 11
distance_between_words = 15
found_64 = 0
for i, img_path in enumerate(X):
  # Reading Image
  candidate_image = cv.imread(dataset_dir + "BD_DB_64/raw/" + img_path)
  # Function Call
  candidate_rs = resize_image(candidate_image)
  candidate_edged = preprocess_image(candidate_rs)
  dilated = transform_image(distance_between_lines, distance_between_words, candidate_edged)
  # dilated_lines = dilated[0]
  dilated_words = dilated[1]
  # segment_lines(img_path, candidate_rs, dilated_lines)
  # segment_words(found_64, img_path, candidate_rs, dilated_words)
  found_64 = segment_words(found_64, img_path, candidate_rs, dilated_words)
print("Found 64 words in", found_64,"images out of", X.shape[0],"images.")

AC_01.jpg (Word): 64
AB_01.jpg (Word): 64
AB_03.jpg (Word): 64
AB_04.jpeg (Word): 64
AB_05.jpg (Word): 64
AC_02.jpg (Word): 64
AC_04.jpg (Word): 64
AC_05.jpg (Word): 64
AH_1.jpg (Word): 64
AH_2.jpg (Word): 64
AH_3.jpg (Word): 64
AH_4.jpg (Word): 64
AH_5.jpg (Word): 64
AH_6.jpg (Word): 64
AH_7.jpg (Word): 64
AH_8.jpg (Word): 64
AH_9.jpg (Word): 64
AH_11.jpg (Word): 64
AH_12.jpg (Word): 64
AH_13.jpg (Word): 64
AH_14.jpg (Word): 64
AH_15.jpg (Word): 64
AH_16.jpg (Word): 64
AH_17.jpg (Word): 64
AH_18.jpg (Word): 64
AH_19.jpg (Word): 64
Found 64 words in 26 images out of 26 images.
